Trabalhando com arquivos
===

Em Python é fácil trabalharmos com arquivos.  Os principais métodos que usamos são `open()`, `close()`, `read()` e `write()`, com as funções óbvias que os nomes informam.

In [ ]:
# Observe o uso implícito do método read() no loop for abaixo
from itertools import islice

f = open("arquivos.ipynb", "rt")
for line in islice(f, 10):
    print(line, end="")
f.close()

In [ ]:
# Para escrever em um arquivo basta usarmos o método write()
f1 = open("arquivos.ipynb", "rt")
f2 = open("arquivos_copia.ipynb", "wt")
for line in f1:
    f2.write(line)
f1.close()
f2.close()

Observe que existe a necessidade de fechar o arquivo ao final do processamento, independente se o processamento foi bem sucedido ou não.  Esta é o objetivo do método `close()`.  Esta tarefa pode ser automatizada através do uso de `with`.

In [ ]:
with open("arquivos.ipynb", "rt") as f1:
    with open("arquivos_copia.ipynb", "wt") as f2:
        for line in f1:
            f2.write(line)

Gerenciadores de contexto
==

A instrução `with` pode ser usado com gerenciadores de contexto, que nada mais são do que objetos contendo dois métodos, `__enter__()` e `__exit__(exc_type, exc_val, exc_tb)`.  Os argumentos deste último método referem-se aos dados de qualquer exceção que ocorra durante a execução do corpo da instrução `with`.  Se nenhuma exceção tiver ocorrido, os três parâmetros tem o valor `None`.

O valor de retorno do método `__enter__()` é atribuído à variável que segue a cláusula `as` da instrução `with`.  O valor booleano de retorno do método `__exit__()` é usado para indicar se a exceção ocorrida no corpo de `with` deve ser suprimida (no caso de retornar `True`) ou não.

In [ ]:
# Exemplo simples de um gerenciador de contexto
class My_Context:
    def __enter__(self):
        print("Entrando no gerenciador de contexto")
        return []
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if not exc_type:
            print("Saindo do gerenciador de contexto sem ter ocorrido exceção")
            return True
        print("Saindo do gerenciador de contexto tendo ocorrido a exceção do tipo", exc_type)
        if exc_type == RuntimeError:
            return False
        else:
            return True

with My_Context() as m:
    m.append(1)
    print(m)

with My_Context() as m:
    raise ValueError("Tentativa 1")
    
try:
    with My_Context() as m:
        raise RuntimeError("Tentativa 2")
except RuntimeError as e:
    print("ERROR: Exceção capturada:", e)

Podemos combinar um gerador, visto na última aula, com um tipo especial de decorador para criar um gerenciador de contexto facilmente.  Façamos um parênteses aqui para explicar o conceito de decorador.

Decoradores
===

Em Python, um *decorador* nada mais é do que uma função que aceita uma função como argumento e retorna uma função.  A ideia é que o decorador realize alguma tarefa antes de chamar a função original em si.

In [ ]:
def decorator(f):
    def f_ret(*args):
        print("Tarefa a realizar antes de chamar a função f")
        res = f(*args)
        print("Tarefa a realizar depois de chamar a função f")
        return res
    return f_ret

@decorator
def g(a, b):
    print("Realizando a tarefa original...")
    print(a, b)
    print("Terminada a tarefa original")
    return "É o fim..."

res = g(10, 20)
print(res)

** Exercícios **

Crie um decorador usando a função `time` do módulo `time` para medir o tempo de execução de uma função qualquer.  Aplique-o a uma função sua.

In [ ]:
import time

def decorator(f):
    def f_ret(*args):
        t1 = time.time()
        res = f(*args)
        t2 = time.time()
        return res, t2 - t1
    return f_ret

@decorator
def g():
    time.sleep(1)

print(g())

Gerenciadores de contexto (parte 2)
==

Visto decoradores, estudemos sua aplicação para geração de gerenciadores de contexto.

Nesse exemplo, declaramos um tipo especial de gerador e o decoramos com um tipo especial de decorador definido por `contextlib.contextmanager`.  Este decorador, após criar o gerador, o chama **exatamente 1 vez**.  A criação do gerador se dá antes de entrar no bloco da instrução `with`, no método `__enter__()` do `contextmanager`.  Pelo que sabemos do gerador, isto executará as instruções do gerador até a primeira instrução `yield`.  O valor retornado por `__enter__()` é então o valor retornado por essa instrução `yield`.

A chamada do gerador se dá no método `__exit__()`.  Isto executa as instruções do gerador que seguem ao primeiro `yield` e que vão até o segundo `yield` ou até o final do gerador, o que vier primeiro.

Podemos converter o exemplo acima do gerenciador `My_Context()` como segue abaixo.

In [ ]:
from contextlib import contextmanager

@contextmanager
def My_Context():
    print("Entrando no gerenciador de contexto")
    try:
        yield []
    except:
        print("Saindo do gerenciador de contexto tendo ocorrido uma exceção")
    else:
        print("Saindo do gerenciador de contexto sem ter ocorrido exceção")
    
with My_Context() as m:
    pass

with My_Context() as m:
    raise ValueError()

Observe que neste tipo de construção devemos gerenciar as exceções manualmente.  O decorador `contextmanager` lança qualquer exceção ocorrida no corpo de `with` no início de seu método `__exit__()`.